# Hyperparameter Finetuning

We want to find the right parameters for the Generator Network.

By using the best values for the following parameters:

- Number of Epochs (meaning `num_epochs` and `num_steps`)
- Learning Rate
- Batch Size
- Number of Noise Batches
- Number of Layers
- Regularization term
- Number of Neurons for each Network

In [5]:
from src.fyemu_tunable import main, evaluate
import torch
import os
import torchvision.transforms as tt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import optuna
import random
from typing import Dict
from torchvision.models import resnet18

from src.metrics import kl_divergence_between_models

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [21]:
def load_models_dict(path: str="data/new/models") -> Dict[str, torch.nn.Module]:
    de = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = resnet18(num_classes = 10).to(de)
    
    # load all the models
    md = {}
    for list in os.listdir(path):
        
        model.load_state_dict(torch.load(f=os.path.join(path, list), map_location=DEVICE, weights_only=True))
        model.eval()
        md[len(md)] = model

    return md

Check out this little tutorial, to see how we handle the optimization using save states:

https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/001_rdb.html

In [7]:
import logging
import sys
import pickle
import optuna

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "GeneratorOpti2"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

if os.path.exists("sampler2.pkl"):
    restored_sampler = pickle.load(open("sampler2.pkl", "rb"))
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, sampler=restored_sampler)
else:
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)

[I 2025-01-20 09:03:39,593] Using an existing study with name 'GeneratorOpti2' instead of creating a new one.


Using an existing study with name 'GeneratorOpti2' instead of creating a new one.
Using an existing study with name 'GeneratorOpti2' instead of creating a new one.


In [8]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Steps = trial.suggest_int('opt_Steps', 1, 20)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 512)
    opt_Number_of_Noise_Batches = trial.suggest_int('opt_Number_of_Noise_Batches', 1, 10)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)

    # print(f"Epochs: {opt_Epochs} |\nSteps: {opt_Steps} |\nLearning Rate: {opt_Learning_Rate} |\nBatch Size: {opt_Batch_Size} |\nNoise Batches: {opt_Number_of_Noise_Batches} |\nRegularization Term: {opt_Regularization_term} |\nNoise Dim: {opt_Noise_Dim}")
    n_layers = trial.suggest_int('n_layers', 1, 8)

    Layers = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,]
    Layers = Layers[:n_layers]
    # print("Layers: ", Layers)

    mod = main(
        t_Epochs = opt_Epochs,
        t_Steps= opt_Steps,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_Number_of_Noise_Batches = opt_Number_of_Noise_Batches,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        new_baseline=False,
        logs=False,
        model_eval_logs=False,
    )
    
    data_dir = f'data{os.sep}cifar10'

    transform_test = tt.Compose([
        tt.ToTensor(),
        tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    valid_ds = ImageFolder(data_dir+f'{os.sep}test', transform_test)
    valid_dl = DataLoader(valid_ds, 256,)

    exact = resnet18(num_classes = 10)
    n = random.randint(0, len(os.listdir("data/retrain/models"))-1)
    exact.load_state_dict(torch.load(f"data/retrain/models/ResNET18_CIFAR10_RETRAIN_CLASSES_{n}.pt", map_location=DEVICE, weights_only=True))
    div = kl_divergence_between_models(
        model1 = exact,
        model2 = mod,
        data_loader = valid_dl,
    )

    return div

study.optimize(objective, n_trials=5)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler2.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)

[I 2025-01-20 09:04:07,066] Trial 101 finished with value: 2.5965034320950506 and parameters: {'opt_Epochs': 2, 'opt_Steps': 8, 'opt_Learning_Rate': 0.0370144068163447, 'opt_Batch_Size': 89, 'opt_Number_of_Noise_Batches': 10, 'opt_Regularization_term': 0.21092407384220052, 'opt_Noise_Dim': 307, 'n_layers': 4}. Best is trial 96 with value: 2.2524891793727884.


Trial 101 finished with value: 2.5965034320950506 and parameters: {'opt_Epochs': 2, 'opt_Steps': 8, 'opt_Learning_Rate': 0.0370144068163447, 'opt_Batch_Size': 89, 'opt_Number_of_Noise_Batches': 10, 'opt_Regularization_term': 0.21092407384220052, 'opt_Noise_Dim': 307, 'n_layers': 4}. Best is trial 96 with value: 2.2524891793727884.
Trial 101 finished with value: 2.5965034320950506 and parameters: {'opt_Epochs': 2, 'opt_Steps': 8, 'opt_Learning_Rate': 0.0370144068163447, 'opt_Batch_Size': 89, 'opt_Number_of_Noise_Batches': 10, 'opt_Regularization_term': 0.21092407384220052, 'opt_Noise_Dim': 307, 'n_layers': 4}. Best is trial 96 with value: 2.2524891793727884.


[I 2025-01-20 09:04:31,996] Trial 102 finished with value: 2.6940812960267078 and parameters: {'opt_Epochs': 1, 'opt_Steps': 13, 'opt_Learning_Rate': 0.03218001149978253, 'opt_Batch_Size': 205, 'opt_Number_of_Noise_Batches': 7, 'opt_Regularization_term': 0.23291959264837864, 'opt_Noise_Dim': 437, 'n_layers': 2}. Best is trial 96 with value: 2.2524891793727884.


Trial 102 finished with value: 2.6940812960267078 and parameters: {'opt_Epochs': 1, 'opt_Steps': 13, 'opt_Learning_Rate': 0.03218001149978253, 'opt_Batch_Size': 205, 'opt_Number_of_Noise_Batches': 7, 'opt_Regularization_term': 0.23291959264837864, 'opt_Noise_Dim': 437, 'n_layers': 2}. Best is trial 96 with value: 2.2524891793727884.
Trial 102 finished with value: 2.6940812960267078 and parameters: {'opt_Epochs': 1, 'opt_Steps': 13, 'opt_Learning_Rate': 0.03218001149978253, 'opt_Batch_Size': 205, 'opt_Number_of_Noise_Batches': 7, 'opt_Regularization_term': 0.23291959264837864, 'opt_Noise_Dim': 437, 'n_layers': 2}. Best is trial 96 with value: 2.2524891793727884.


[I 2025-01-20 09:05:03,971] Trial 103 finished with value: 2.6948665171861657 and parameters: {'opt_Epochs': 2, 'opt_Steps': 14, 'opt_Learning_Rate': 0.18526503258932397, 'opt_Batch_Size': 150, 'opt_Number_of_Noise_Batches': 5, 'opt_Regularization_term': 0.2240879130770853, 'opt_Noise_Dim': 406, 'n_layers': 5}. Best is trial 96 with value: 2.2524891793727884.


...

Trial 398 finished with value: 2.6925140336155895 and parameters: {'opt_Epochs': 6, 'opt_Steps': 13, 'opt_Learning_Rate': 0.22387382988511675, 'opt_Batch_Size': 460, 'opt_Number_of_Noise_Batches': 1, 'opt_Regularization_term': 0.16234303325055657, 'opt_Noise_Dim': 320, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.
Trial 398 finished with value: 2.6925140336155895 and parameters: {'opt_Epochs': 6, 'opt_Steps': 13, 'opt_Learning_Rate': 0.22387382988511675, 'opt_Batch_Size': 460, 'opt_Number_of_Noise_Batches': 1, 'opt_Regularization_term': 0.16234303325055657, 'opt_Noise_Dim': 320, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.


[I 2025-01-20 12:47:31,504] Trial 399 finished with value: 2.5482951045036315 and parameters: {'opt_Epochs': 6, 'opt_Steps': 13, 'opt_Learning_Rate': 0.21995610302495325, 'opt_Batch_Size': 487, 'opt_Number_of_Noise_Batches': 1, 'opt_Regularization_term': 0.08665392492392385, 'opt_Noise_Dim': 282, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.


Trial 399 finished with value: 2.5482951045036315 and parameters: {'opt_Epochs': 6, 'opt_Steps': 13, 'opt_Learning_Rate': 0.21995610302495325, 'opt_Batch_Size': 487, 'opt_Number_of_Noise_Batches': 1, 'opt_Regularization_term': 0.08665392492392385, 'opt_Noise_Dim': 282, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.
Trial 399 finished with value: 2.5482951045036315 and parameters: {'opt_Epochs': 6, 'opt_Steps': 13, 'opt_Learning_Rate': 0.21995610302495325, 'opt_Batch_Size': 487, 'opt_Number_of_Noise_Batches': 1, 'opt_Regularization_term': 0.08665392492392385, 'opt_Noise_Dim': 282, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.


[I 2025-01-20 12:48:56,745] Trial 400 finished with value: 2.4003746405243875 and parameters: {'opt_Epochs': 6, 'opt_Steps': 14, 'opt_Learning_Rate': 0.21368939022706987, 'opt_Batch_Size': 477, 'opt_Number_of_Noise_Batches': 2, 'opt_Regularization_term': 0.14801614946380903, 'opt_Noise_Dim': 308, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.


Trial 400 finished with value: 2.4003746405243875 and parameters: {'opt_Epochs': 6, 'opt_Steps': 14, 'opt_Learning_Rate': 0.21368939022706987, 'opt_Batch_Size': 477, 'opt_Number_of_Noise_Batches': 2, 'opt_Regularization_term': 0.14801614946380903, 'opt_Noise_Dim': 308, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.
Trial 400 finished with value: 2.4003746405243875 and parameters: {'opt_Epochs': 6, 'opt_Steps': 14, 'opt_Learning_Rate': 0.21368939022706987, 'opt_Batch_Size': 477, 'opt_Number_of_Noise_Batches': 2, 'opt_Regularization_term': 0.14801614946380903, 'opt_Noise_Dim': 308, 'n_layers': 3}. Best is trial 276 with value: 2.1014704927802086.


In [10]:
trials_df = study.trials_dataframe()
best10_df = trials_df.sort_values("value").head(10)
best10_df

,number,value,datetime_start,datetime_complete,duration,params_n_layers,params_opt_Batch_Size,params_opt_Epochs,params_opt_Learning_Rate,params_opt_Noise_Dim,params_opt_Number_of_Noise_Batches,params_opt_Regularization_term,params_opt_Steps,state
276,276,2.101470,2025-01-20 10:48:05.124164,2025-01-20 10:48:59.893961,0 days 00:00:54.769797,2,367,5,0.221830,81,1,0.195540,16,COMPLETE
384,384,2.161381,2025-01-20 12:28:15.308205,2025-01-20 12:29:23.575110,0 days 00:01:08.266905,2,456,6,0.204652,264,1,0.164976,13,COMPLETE
296,296,2.166698,2025-01-20 11:08:35.864121,2025-01-20 11:09:31.924229,0 days 00:00:56.060108,2,367,6,0.223592,76,2,0.154956,14,COMPLETE
355,355,2.172314,2025-01-20 12:04:09.810198,2025-01-20 12:04:59.241561,0 days 00:00:49.431363,2,366,5,0.224452,40,1,0.126115,14,COMPLETE
301,301,2.173893,2025-01-20 11:13:27.593964,2025-01-20 11:14:25.961665,0 days 00:00:58.367701,2,370,6,0.229829,84,1,0.155063,15,COMPLETE
309,309,2.180563,2025-01-20 11:21:49.752380,2025-01-20 11:22:49.560179,0 days 00:00:59.807799,2,405,6,0.220118,56,1,0.160018,14,COMPLETE
397,397,2.191130,2025-01-20 12:43:30.132685,2025-01-20 12:44:49.503025,0 days 00:01:19.370340,3,467,6,0.221574,320,1,0.160935,13,COMPLETE
284,284,2.200116,2025-01-20 10:55:34.314130,2025-01-20 10:56:38.569440,0 days 00:01:04.255310,2,390,6,0.218416,419,2,0.169877,15,COMPLETE
250,250,2.200357,2025-01-20 10:27:50.965939,2025-01-20 10:28:29.385641,0 days 00:00:38.419702,4,316,3,0.266277,115,1,0.189250,12,COMPLETE
363,363,2.200668,2025-01-20 12:10:39.588182,2025-01-20 12:11:22.462954,0 days 00:00:42.874772,2,350,4,0.135425,71,1,0.118336,14,COMPLETE


^^^This will represent the values used as default^^^

In [9]:
study.best_params

{'opt_Epochs': 5,
 'opt_Steps': 16,
 'opt_Learning_Rate': 0.22183049596711268,
 'opt_Batch_Size': 367,
 'opt_Number_of_Noise_Batches': 1,
 'opt_Regularization_term': 0.19553973560496515,
 'opt_Noise_Dim': 81,
 'n_layers': 2}

___

### Standard Parameters

In [14]:
standard_model = main(
    t_Epochs = study.best_params["opt_Epochs"],
    t_Steps= study.best_params["opt_Steps"],
    t_Learning_Rate = study.best_params["opt_Learning_Rate"],
    t_Batch_Size = study.best_params["opt_Batch_Size"],
    t_Number_of_Noise_Batches = study.best_params["opt_Number_of_Noise_Batches"],
    t_Regularization_term = study.best_params["opt_Regularization_term"],
    t_Layers = [1000, 1000, 1000,1000,1000,1000,1000,1000,1000,1000,][:study.best_params["n_layers"]],
    t_Noise_Dim = study.best_params["opt_Noise_Dim"],
    new_baseline=False,
    logs=True,
    model_eval_logs=True
)

['test', 'train']
['bird', 'deer', 'horse', 'automobile', 'frog', 'airplane', 'truck', 'cat', 'dog', 'ship']
[{'Loss': 1.22891104221344, 'Acc': 0.77801114320755}]
---Optimizing noise generator---
Optiming loss for class 0
Loss: 24297.083984375
Loss: 4410.2900390625
Loss: 987.8056640625
Loss: 266.85516357421875
Loss: 110.88993072509766
Optiming loss for class 2
Loss: 27758.142578125
Loss: 5074.34423828125
Loss: 1137.2779541015625
Loss: 298.7597351074219
Loss: 117.35765075683594
---Impairing Phase---
Train loss 1: 0.16312381982922555,Train Acc:12.868%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.0
Loss: 8.782315254211426
Performance of Standard Forget Model on Retain Class
Accuracy: 65.33447504043579
Loss: 0.9652000069618225
---Repairing Phase---
Train loss 1: 0.08421937516093254,Train Acc:13.07%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.0
Loss: 9.555630683898926
Performance of Standard Forget Model on Retain Class
Accuracy: 74.36279058456421
Lo

___

In [22]:
from torchvision.models import resnet18
import torch

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

train_ms = load_models_dict(path="data/all/models")

exact_ms = load_models_dict(path="data/retrain/models")

In [24]:
from src.metrics import kl_divergence_between_models
import os
import torchvision.transforms as tt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

batch_size = 256
data_dir = f'data{os.sep}cifar10'

transform_test = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

valid_ds = ImageFolder(data_dir+f'{os.sep}test', transform_test)
valid_dl = DataLoader(valid_ds, batch_size, shuffle=False)

kl_divergence_between_models(model1 = train_ms[0], model2 = train_ms[0], data_loader = valid_dl)

8.873657390481847e-11

In [25]:
kl_divergence_between_models(model1=exact_ms[0], model2=exact_ms[0], data_loader=valid_dl)

4.918620909871717e-11

In [27]:
kl_divergence_between_models(model1=exact_ms[0], model2=train_ms[0], data_loader=valid_dl)

3.0422201290726663

In [28]:
kl_divergence_between_models(model1=exact_ms[0], model2=standard_model, data_loader=valid_dl)

2.1034967750310907